<center>
<h4>CDS 110, Lecture 3</h4>
<font color=blue><h1>Python Tools for Analyzing Linear Systems</h1></font>
<h3>Richard M. Murray, Winter 2024</h3>
</center>

[Open in Google Colab](https://colab.research.google.com/drive/164yYvB86c2EvEcIHpUPNXCroiN9nnTAa)

In this lecture we describe tools in the Python Control Systems Toolbox ([python-control](https://python-control.org)) that can be used to analyze linear systems, including some of the options available to present the information in different ways.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
try:
  import control as ct
  print("python-control", ct.__version__)
except ImportError:
  !pip install control
  import control as ct

## Coupled mass spring system

Consider the spring mass system below:

<center><img src="https://www.cds.caltech.edu/~murray/courses/cds110/sp2024/springmass-coupled.png" width=640></center>

We wish to analyze the time and frequency response of this system using a variety of python-control functions for linear systems analysis.

### System dynamics

The dynamics of the system can be written as

$$
\begin{aligned}
  m \ddot{q}_1 &= -2 k q_1 - c \dot{q}_1 + k q_2, \\
  m \ddot{q}_2 &= k q_1 - 2 k q_2 - c \dot{q}_2 + ku
\end{aligned}
$$

or in state space form:

$$
\begin{aligned}
  \dfrac{dx}{dt} &= \begin{bmatrix}
    0 & 0 & 1 & 0 \\
    0 & 0 & 0 & 1 \\[0.5ex]
    -\dfrac{2k}{m} & \dfrac{k}{m} & -\dfrac{c}{m} & 0 \\[0.5ex]
    \dfrac{k}{m} & -\dfrac{2k}{m} & 0 & -\dfrac{c}{m}
  \end{bmatrix} x
  + \begin{bmatrix}
    0 \\ 0 \\[0.5ex] 0 \\[1ex] \dfrac{k}{m}
  \end{bmatrix} u.
\end{aligned}
$$



In [ ]:
# Define the parameters for the system
m, c, k = 1, 0.1, 2
# Create a linear system
A = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [-2*k/m, k/m, -c/m, 0],
    [k/m, -2*k/m, 0, -c/m]
])
B = np.array([[0], [0], [0], [k/m]])
C = np.array([[1, 0, 0, 0], [0, 1, 0, 0]])
D = 0

sys = ct.ss(A, B, C, D, outputs=['q1', 'q2'], name="coupled spring mass")
print(sys)

Another way to get these same dynamics is to define an input/output system:

In [ ]:
coupled_params = {'m': 1, 'c': 0.1, 'k': 2}
def coupled_update(t, x, u, params):
  m, c, k = params['m'], params['c'], params['k']
  return np.array([
      x[2], x[3],
      -2*k/m * x[0] + k/m * x[1] - c/m * x[2],
      k/m * x[0] -2*k/m * x[1] - c/m * x[3] + k/m * u[0]
  ])
def coupled_output(t, x, u, params):
  return x[0:2]
coupled = ct.nlsys(
    coupled_update, coupled_output, inputs=1, outputs=['q1', 'q2'],
    states=['q1', 'q2', 'q1dot', 'q2dot'], name='coupled (nl)',
    params=coupled_params
)
print(coupled.linearize([0, 0, 0, 0], [0]))

### Initial response

The `initial_response` function can be used to compute the response of the system with no input, but starting from a given initial condition.  This function returns a response object, which can be used for plotting.

In [ ]:
response = ct.initial_response(sys, X0=[1, 0, 0, 0])
cplt = response.plot()

If you want to play around with the way the data are plotted, you can also use the response object to get direct access to the states and outputs.

In [ ]:
# Plot the outputs of the system on the same graph, in different colors
t = response.time
x = response.states
plt.plot(t, x[0], 'b', t, x[1], 'r')
plt.legend(['$x_1$', '$x_2$'])
plt.xlim(0, 50)
plt.ylabel('States')
plt.xlabel('Time [s]')
plt.title("Initial response from $x_1 = 1$, $x_2 = 0$");

There are also lots of options available in `initial_response` and `.plot()` for tuning the plots that you get.

In [ ]:
for X0 in [[1, 0, 0, 0], [0, 2, 0, 0], [1, 2, 0, 0], [0, 0, 1, 0], [0, 0, 2, 0]]:
  response = ct.initial_response(sys, T=20, X0=X0)
  response.plot(label=f"{X0=}")

### Step response

Similar to `initial_response`, you can also generate a step response for a linear system using the `step_response` function, which returns a time  response object:

In [ ]:
cplt = ct.step_response(sys).plot()

We can analyze the properties of the step response using the `stepinfo` command:

In [ ]:
step_info = ct.step_info(sys)
print("Input 0, output 0 rise time = ",
      step_info[0][0]['RiseTime'], "seconds\n")
step_info

Note that by default the inputs are not included in the step response plot (since they are a bit boring), but you can change that:

In [ ]:
stepresp = ct.step_response(sys)
cplt = stepresp.plot(plot_inputs=True)

In [ ]:
# Plot the inputs on top of the outputs
cplt = stepresp.plot(plot_inputs='overlay')

In [ ]:
# Look at the "shape" of the step response
print(f"{stepresp.time.shape=}")
print(f"{stepresp.inputs.shape=}")
print(f"{stepresp.states.shape=}")
print(f"{stepresp.outputs.shape=}")

## Forced response

To compute the response to an input, using the convolution equation, we can use the `forced_response` function:

In [ ]:
T = np.linspace(0, 50, 500)
U1 = np.cos(T)
U2 = np.sin(3 * T)

resp1 = ct.forced_response(sys, T, U1)
resp2 = ct.forced_response(sys, T, U2)
resp3 = ct.forced_response(sys, T, U1 + U2)

# Plot the individual responses
resp1.sysname = 'U1'; resp1.plot(color='b')
resp2.sysname = 'U2'; resp2.plot(color='g')
resp3.sysname = 'U1 + U2'; resp3.plot(color='r');

In [ ]:
# Show that the system response is linear
cplt = resp3.plot()
cplt.axes[0, 0].plot(resp1.time, resp1.outputs[0] + resp2.outputs[0], 'k--')
cplt.axes[1, 0].plot(resp1.time, resp1.outputs[1] + resp2.outputs[1], 'k--')
cplt.axes[2, 0].plot(resp1.time, resp1.inputs[0] + resp2.inputs[0], 'k--');

In [ ]:
# Show that the forced response from non-zero initial condition is not linear
X0 = [1, 0, 0, 0]
resp1 = ct.forced_response(sys, T, U1, X0=X0)
resp2 = ct.forced_response(sys, T, U2, X0=X0)
resp3 = ct.forced_response(sys, T, U1 + U2, X0=X0)

cplt = resp3.plot()
cplt.axes[0, 0].plot(resp1.time, resp1.outputs[0] + resp2.outputs[0], 'k--')
cplt.axes[1, 0].plot(resp1.time, resp1.outputs[1] + resp2.outputs[1], 'k--')
cplt.axes[2, 0].plot(resp1.time, resp1.inputs[0] + resp2.inputs[0], 'k--');

### Frequency response

In [ ]:
# Manual computation of the frequency response
resp = ct.input_output_response(sys, T, np.sin(1.35 * T))

cplt = resp.plot(
    plot_inputs='overlay', 
    legend_map=np.array([['lower left'], ['lower left']]),
    label=[['q1', 'u[0]'], ['q2', None]])

The magnitude and phase of the frequency response is controlled by the transfer function,

$$
G(s) = C (sI - A)^{-1} B + D
$$

which can be computed using the `ss2tf` function:

In [ ]:
try:
    G = ct.ss2tf(sys, name='u to q1, q2')
except ct.ControlMIMONotImplemented:
    # Create SISO transfer functions, in case we don't have slycot
    G = ct.ss2tf(sys[0, 0], name='u to q1')
print(G)

In [ ]:
# Gain and phase for the simulation above
from math import pi
val = G(1.35j)
print(f"{G(1.35j)=}")
print(f"Gain: {np.absolute(val)}")
print(f"Phase: {np.angle(val)}", " (", np.angle(val) * 180/pi, "deg)")

In [ ]:
# Gain and phase at s = 0 (= steady state step response)
print(f"{G(0)=}")
print("Final value of step response:", stepresp.outputs[0, 0, -1])

The frequency response across all frequencies can be computed using the `frequency_response` function:

In [ ]:
freqresp = ct.frequency_response(sys)
cplt = freqresp.plot()

By default, frequency responses are plotted using a "Bode plot", which plots the log of the magnitude and the (linear) phase against the log of the forcing frequency.

You can also call the Bode plot command directly, and change the way the data are presented:

In [ ]:
cplt = ct.bode_plot(sys, overlay_outputs=True)

Note the "dip" in the frequency response for y[1] at frequency 2 rad/sec, which corresponds to a "zero" of the transfer function.

This dip becomes even more pronounced in the case of low damping coefficient $c$:

In [ ]:
cplt = ct.frequency_response(
    coupled.linearize([0, 0, 0, 0], [0], params={'c': 0.01})
).plot(overlay_outputs=True)

## Additional resources
* [Code for FBS2e figures](https://fbswiki.org/wiki/index.php/Category:Figures): Python code used to generate figures in FBS2e
* [Python-control documentation for plotting time responses](https://python-control.readthedocs.io/en/0.10.0/plotting.html#time-response-data)
* [Python-control documentation for plotting frequency responses](https://python-control.readthedocs.io/en/0.10.0/plotting.html#frequency-response-data)
* [Python-control examples](https://python-control.readthedocs.io/en/0.10.0/examples.html): lots of Python and Jupyter examples of control system analysis and design
